# Produção dos gráficos de Usuários vs. Indicadores

A partir de dados coletados pela API do Twitter, o código a seguir permite a extração do número de usuários únicos ativos ao longo das semanas epidemiológicas.
Então, esses dados são cruzados com os indicadores para produzir gráficos comparativos.

In [48]:
import pandas as pd
import os
from datetime import datetime

### Obtendo número de usuários

In [77]:
path = os.getcwd() + '\\results\\results_weekly\\week_'
filename = 'tweets_users_'
unique_users = pd.DataFrame(columns=['total_users', 'tweets_users', 'retweets_users'])

for i in range(2,38):
    print('Reading week', i)
    tempDf = pd.read_csv(path + str(i) + '\\' + filename + str(i) + '.csv', delimiter=';', usecols=['user_id'])
    rtTempDf = pd.read_csv(path + str(i) + '\\rt_' + filename + str(i) + '.csv', delimiter=';', usecols=['user_id'])
    
    auxDf = pd.concat([tempDf, rtTempDf])
    
    unique_users = unique_users.append({
        'total_users': len(auxDf['user_id'].unique()),
        'tweets_users': len(tempDf['user_id'].unique()),
        'retweets_users': len(rtTempDf['user_id'].unique())
    }, ignore_index=True)

Reading week 2
Reading week 3
Reading week 4
Reading week 5
Reading week 6
Reading week 7
Reading week 8
Reading week 9
Reading week 10
Reading week 11
Reading week 12
Reading week 13
Reading week 14
Reading week 15
Reading week 16
Reading week 17
Reading week 18
Reading week 19
Reading week 20
Reading week 21
Reading week 22
Reading week 23
Reading week 24
Reading week 25
Reading week 26
Reading week 27
Reading week 28
Reading week 29
Reading week 30
Reading week 31
Reading week 32
Reading week 33
Reading week 34
Reading week 35
Reading week 36
Reading week 37


### Anexando N. de Usuários ao DataFrame de Tweets

In [78]:
tweets = pd.read_csv('tweetsAndRetweets.csv')

In [80]:
for column in unique_users.columns:
    tweets[column] = unique_users[column]

### Lendo e tratando indicadores

In [83]:
indicadores = pd.read_csv('indicadores.csv')

indicadores['week_last_day'] = indicadores['week_last_day'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y')) # Convertendo os dias de String para Datetime
indicadores = indicadores.loc[indicadores['region'] == 'Brasil']
indicadores = indicadores[indicadores['week_last_day'] >= pd.to_datetime('2020-05-09')] # Cortando primeiras semanas para parear com o "df"
indicadores = indicadores.sort_values(by='week_last_day')
indicadores = indicadores.reset_index()
indicadores = indicadores.drop(columns=['index', 'Unnamed: 0', 'region', 'week_last_day', 'epidemiological_week'])

### Anexando indicadores ao Dataframe de Tweets e Usuários

In [84]:
for column in indicadores.columns:
    tweets[column] = indicadores[column]

In [85]:
tweets.to_csv('tweets_and_indicators.csv', index=False)

Descartando a última semana, que contém dados incompletos

In [97]:
df = tweets[:-1]

## Plotando gráficos

In [110]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Razão <i>Tweets</i>/<i>Retweets</i> por semana

In [116]:
fig = make_subplots()
fig.add_trace(go.Scatter(y=df['tweets']/df['retweets'], x=df['date'],  name='<i>Tweets/Retweets</i> por semana'))

# Add figure title
fig.update_layout(
    title_text="Razão <i>Tweets</i>/<i>Retweets</i> por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="<i>Tweets</i>/Retweets por semana", range=[0,1])

fig.write_html('html/tweets_over_retweets.html')
fig.write_image('svg/tweets_over_retweets.svg', width=1280, height=720)

#### Razão <i>Tweets</i>/<i>Usuários</i> por semana

In [111]:
fig = make_subplots()
fig.add_trace(go.Scatter(y=df['tweets']/df['tweets_users'], x=df['date'],  name='<i>Tweets</i>/Usuários por semana'))

# Add figure title
fig.update_layout(
    title_text="Razão <i>Tweets</i>/Usuários por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="<i>Tweets</i>/Usuários por semana", range=[0,5])

fig.write_html('html/tweets_over_users.html')
fig.write_image('svg/tweets_over_users.svg', width=1280, height=720)

#### Razão <i>Reweets</i>/<i>Usuários</i> por semana

In [112]:
fig = make_subplots()
fig.add_trace(go.Scatter(y=df['retweets']/df['retweets_users'], x=df['date'],  name='<i>Retweets</i>/Usuários por semana'))

# Add figure title
fig.update_layout(
    title_text="Razão <i>Retweets</i>/Usuários por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="<i>Retweets</i>/Usuários por semana", range=[0,5])

fig.write_html('html/retweets_over_users.html')
fig.write_image('svg/retweets_over_users.svg', width=1280, height=720)

#### Comparação da razão <i>Tweets</i> vs. <i>Reweets</i>/<i>Usuários</i> por semana

In [113]:
fig = make_subplots()
fig.add_trace(go.Scatter(y=df['retweets']/df['retweets_users'], x=df['date'],  name='<i>Retweets</i>/Usuários por semana'))
fig.add_trace(go.Scatter(y=df['tweets']/df['tweets_users'], x=df['date'],  name='<i>Tweets</i>/Usuários por semana'))


# Add figure title
fig.update_layout(
    title_text="Comparação da razão <i>Retweets</i>/<i>Usuários</i> por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="<i>Tweets</i>/Usuários por semana", range=[0,5])

fig.write_html('html/tweets_vs_retweets_over_users.html')
fig.write_image('svg/tweets_vs_retweets_over_users.svg', width=1280, height=720)

#### Razão <i>Total de Tweets</i>/<i>Usuários</i> por semana

In [114]:
fig = make_subplots()
fig.add_trace(go.Scatter(y=df['total']/df['total_users'], x=df['date'],  name='Total de <i>Tweets</i>/Usuários por semana'))

# Add figure title
fig.update_layout(
    title_text="Razão Total de <i>Tweets</i>/Usuários por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="Total de <i>Tweets</i>/Usuários por semana", range=[0,5])

fig.write_html('html/total_tweets_over_users.html')
fig.write_image('svg/total_tweets_over_users.svg', width=1280, height=720)

#### Total de <i>Tweets</i> vs. Novos óbitos por semana

In [115]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=df['tweets'], x=df['date'],  name='Total de <i>Tweets</i> por semana'), secondary_y = True)
fig.add_trace(go.Scatter(y=df['new_week_deaths'], x=df['date'],  name='<i>Novos óbitos</i> por semana'), secondary_y=False)


# Add figure title
fig.update_layout(
    title_text="Total de <i>Tweets</i> vs. Novos óbitos por semana"
)
# Set y-axes titles
fig.update_yaxes(title_text="Novos óbitos por semana", secondary_y=False)
fig.update_yaxes(title_text="Total de <i>Tweets</i> por semana", secondary_y=True)

fig.write_html('html/total_tweets_vs_deaths.html')
fig.write_image('svg/total_tweets_vs_deaths.svg', width=1280, height=720)